In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, sys, gc
import random
import math
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
from multiprocessing import cpu_count, current_process
import seaborn as sns
from datetime import datetime
from IPython.display import clear_output

In [3]:
import pickle
import feather
from imblearn.over_sampling import SMOTE
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor, CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier
from kuma_utils.BearBoost import BearBoostStacking
from kuma_utils.ModelTuna import model_tuna, cross_validate
from kuma_utils.PreProcessing import *#adversarial_validation, target_encoding, KMeansFeaturizer, parallel_apply, thread_wrapper

/Users/CL4US/.pyenv/versions/finowcast/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/Users/CL4US/.pyenv/versions/finowcast/lib/python3.6/site-packages/chainer/_environment_check.py:37: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BL

In [4]:
# dir_data = '/mnt/gcs-bucket/kaggle/santander/'
dir_data = 'rawdata'

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
train_data = pd.read_hdf(os.path.join(dir_data, 'train.hdf'))
test_data = pd.read_hdf(os.path.join(dir_data, 'test.hdf'))

In [7]:
public_lb_idx = np.load(os.path.join(dir_data, 'public_LB.npy'))
private_lb_idx = np.load(os.path.join(dir_data, 'private_LB.npy'))
synthesized_idx = np.load(os.path.join(dir_data, 'synthetic_samples_indexes.npy'))

---
# Data processing
## DenoisingAutoEncoder
**Not** working

In [ ]:
train_data = rank_gauss(train_data, skip_cols=['target'])

In [ ]:
noised_train = swap_noise(train_data.drop('target', axis=1).values)
noised_train = noised_train.astype(np.float32)

In [ ]:
import chainer
from chainer import Variable, Chain, optimizers
import chainer.links as L
import chainer.functions as F
from chainer import training, iterators, reporter
from chainer.training import extensions

In [ ]:
class easyAE(Chain):
    def __init__(self):
        super(easyAE, self).__init__(
            l1=L.Linear(len(FEATURE_COL),50),
            l2=L.Linear(50, 50),
            l3=L.Linear(50,len(FEATURE_COL)),
        )
        
    def __call__(self, x):
        y = self.fwd(x)
        loss = F.mean_squared_error(y, x)
        reporter.report({'loss': loss}, self)
        return loss
        
    def fwd(self, x):
        y = F.relu(self.l1(x))
        y = F.dropout(F.relu(self.l2(y)))
        y = self.l3(y)
        return y
    
    def get_features(self, x):
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        return y

In [ ]:
ae = easyAE()
optimizer = optimizers.Adam()
optimizer.setup(ae)

In [ ]:
train_iter = iterators.SerialIterator(noised_train, 128)
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (100, 'epoch'))
trainer.extend(extensions.PrintReport(['iteration', 'main/loss', 'elapsed_time']))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.ProgressBar())
trainer.run()

In [ ]:
# x = Variable(.values)
# new_features = ae.get_features(x).data
# new_features = pd.DataFrame(data=new_features, index=train_data.index, 
#                             columns=['ae_{}'.format(i) for i in range(new_features.shape[1])])

In [ ]:
# x = Variable(test_data.values)
# test_new_features = ae.get_features(x).data
# test_new_features = pd.DataFrame(data=test_new_features, index=test_data.index, 
#                             columns=['ae_{}'.format(i) for i in range(new_features.shape[1])])

In [ ]:
# new_corr = new_features.corr()
# plt.figure(figsize=(20, 10))
# sns.heatmap(new_corr, vmin=-1.0, vmax=1.0, center=0, annot=True, fmt='.1f', 
#             xticklabels=new_corr.columns.values, yticklabels=new_corr.columns.values)

---
## Manual feature engineering

In [6]:
train_data = pd.read_hdf(os.path.join(dir_data, 'train.hdf'))
test_data = pd.read_hdf(os.path.join(dir_data, 'test.hdf'))

In [7]:
FEATURE_COL = test_data.columns

In [67]:
train_data = pd.merge(train_data[['target']], pd.read_hdf(os.path.join(dir_data, 'vert_rank.hdf')), 
                      left_index=True, right_index=True)

In [134]:
train_data = pd.merge(train_data, pd.read_hdf(os.path.join(dir_data, 'cmn_f.hdf')), 
                      left_index=True, right_index=True)
# test_data = pd.merge(test_data, pd.read_hdf(os.path.join(dir_data, 'cnt_f_test.hdf')), 
#                       left_index=True, right_index=True, how='right')

In [68]:
train_data.head()#.corr()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,cnt_var_190_y,cnt_var_191_y,cnt_var_192_y,cnt_var_193_y,cnt_var_194_y,cnt_var_195_y,cnt_var_196_y,cnt_var_197_y,cnt_var_198_y,cnt_var_199_y
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,0.000015,0.00003,0.000035,0.000015,0.000020,0.000020,0.000015,0.000065,0.000025,0.000010
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,0.000025,0.00002,0.000030,0.000005,0.000005,0.000010,0.000010,0.000065,0.000010,0.000005
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,0.000015,0.00002,0.000015,0.000005,0.000010,0.000010,0.000015,0.000040,0.000010,0.000010
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,0.000005,0.00001,0.000020,0.000020,0.000015,0.000035,0.000020,0.000020,0.000010,0.000010
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,0.000015,0.00002,0.000005,0.000005,0.000005,0.000025,0.000015,0.000030,0.000010,0.000010


## Data augmentation (shuffling)
as reported in discussion, shuffleing data inside a column does not affect prediction ;)
**slightly working**

In [104]:
def shuffle_augmentation(x_train, y_train, x_test, y_test, multiplier=(1, 2)):
    data_0 = x_train[y_train == 0]
    data_1 = x_train[y_train == 1]
    new_x_0 = np.empty((0, x_train.shape[1]))
    new_x_1 = np.empty((0, x_train.shape[1]))
    new_y_0 = np.zeros(data_0.shape[0] * multiplier[0])
    new_y_1 = np.ones(data_1.shape[0] * multiplier[1])
    for i in range(multiplier[0]):
        new_0 = np.empty_like(data_0)
        for col in range(x_train.shape[1]):
            new_0[:, col] = np.random.permutation(data_0[:, col])
        new_x_0 = np.concatenate([new_x_0, new_0])
    for i in range(multiplier[1]):
        new_1 = np.empty_like(data_1)
        for col in range(x_train.shape[1]):
            new_1[:, col] = np.random.permutation(data_1[:, col])
        new_x_1 = np.concatenate([new_x_1, new_1])
    print('[Shuffle Augmentation]\ntarget_0: {} -> {}\ntarget_1: {} -> {}'.format(data_0.shape[0], data_0.shape[0]+new_x_0.shape[0],
                                                           data_1.shape[0], data_1.shape[0]+new_x_1.shape[0]))
    return np.concatenate([x_train, new_x_0, new_x_1]), np.concatenate([y_train, new_y_0, new_y_1]), x_test, y_test

In [ ]:
# DO this at the beginning is a huge leak!
# new_train = stratified_shuffle(train_data)
# new_train.shape
# train_data = pd.concat([train_data, new_train])
# train_data.target.value_counts()
# train_data.to_hdf(os.path.join(dir_data, 'train_augmented.hdf', key='data'))
# train_data = train_data.reset_index(drop=True)

## New Features

In [33]:
def shuffle_features(data, cols):
    new_features = pd.DataFrame(index=data.index)
    for icol in tqdm(cols):
        tmp_data = data.iloc[:, icol+1].copy()
        break_point = tmp_data.median()
        first_part =  tmp_data[tmp_data < break_point].copy()
        second_part =  tmp_data[tmp_data >= break_point].copy()
        first_part = pd.Series(np.random.permutation(first_part.values), first_part.index)
        second_part = pd.Series(np.random.permutation(second_part.values), second_part.index)
        new_data = pd.concat([first_part, second_part])
        new_data.name = 'sfl_{}'.format(icol)
        new_features = pd.merge(new_features, new_data, left_index=True, right_index=True)
    return new_features

In [123]:
def unique_features(x_train, y_train, x_test, y_test):
    new_train = np.zeros_like(x_train)
    new_test = np.zeros_like(x_test)
    for icol in tqdm(range(x_train.shape[1]), desc='adding unique value features...'):
        tmp_train = np.zeros(x_train.shape[0])
        tmp_test = np.zeros(x_test.shape[0])
        unique = np.unique(x_train[:, icol][y_train == 1])
        tmp_train[np.isin(x_train[:, icol], unique)] = 1
        tmp_test[np.isin(x_test[:, icol], unique)] = 1
        new_train[:, icol] = tmp_train
        new_test[:, icol] = tmp_test
    print('new_features: {}'.format(x_train.shape[1] + new_train.shape[1]))
    return np.concatenate([x_train, new_train], axis=1), y_train, np.concatenate([x_test, new_test], axis=1), y_test

In [6]:
# def count_features(x_train, y_train, x_test, y_test):
#     new_train = np.zeros_like(x_train)
#     new_test = np.zeros_like(x_test)
#     for icol in tqdm(range(x_train.shape[1]), desc='adding count features...'):
#         tmp_train = np.zeros(x_train.shape[0])
#         tmp_test = np.zeros(x_test.shape[0])
#         cnt_dict = pd.Series(x_train[:, icol])
#         tmp_train[np.isin(x_train[:, icol], unique)] = 1
#         tmp_test[np.isin(x_test[:, icol], unique)] = 1
#         new_train[:, icol] = tmp_train
#         new_test[:, icol] = tmp_test
#     print('new_features: {}'.format(x_train.shape[1] + new_train.shape[1]))
#     return np.concatenate([x_train, new_train], axis=1), y_train, np.concatenate([x_test, new_test], axis=1), y_test

In [12]:
def count_features(train_df, test_df):
    new_train = pd.DataFrame()
    new_test = pd.DataFrame()
    for col in tqdm(train_df.columns):
        if col in ['target']:
            continue
            
        col_all = pd.concat([train_df[col], test_df[col]])
        col_name = col + '_cnt'
        col_count = col_all.value_counts().to_dict()
        
        new_train[col_name] = train_df[col].map(col_count).astype(np.int32)
        new_test[col_name] = test_df[col].map(col_count).astype(np.int32)
    
    return new_train, new_test

In [13]:
cnt_f_train, cnt_f_test = count_features(train_data, test_data.iloc[np.concatenate([public_lb_idx, private_lb_idx])])

In [16]:
train_data = pd.merge(train_data, cnt_f_train, left_index=True, right_index=True)
test_data = pd.merge(test_data, cnt_f_test, left_index=True, right_index=True, how='left')

---
# Learning :)
## Data prep

In [17]:
def AUC(y_true, y_pred, weight=None, get_false=False):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    return auc(fpr, tpr)

In [133]:
train_data = pd.read_hdf(os.path.join(dir_data, 'train.hdf'))
test_data = pd.read_hdf(os.path.join(dir_data, 'test.hdf'))

In [18]:
X, y = train_data.drop(['target'], axis=1), train_data['target']

In [19]:
X.shape

(200000, 400)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [21]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190_cnt,var_191_cnt,var_192_cnt,var_193_cnt,var_194_cnt,var_195_cnt,var_196_cnt,var_197_cnt,var_198_cnt,var_199_cnt
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,3,8,9,4,9,5,5,14,5,2
train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,8,5,9,2,4,4,4,21,6,2
train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,4,5,6,2,2,2,3,12,4,2
train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,2,3,6,4,4,8,5,4,2,2
train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,3,8,1,1,1,9,5,9,2,2


In [22]:
# CAT_FEATURES = ['var_68']
# CAT_FEATURES_IDX = [int(X.columns.get_loc(x)) for x in CAT_FEATURES]
# CON_FEATURES_IDX = list(set([x for x in range(X.shape[1])]) - set(CAT_FEATURES_IDX))

In [23]:
# sc = StandardScaler()
# x_train = pd.DataFrame(sc.fit_transform(x_train), columns=X.columns)
# x_test = pd.DataFrame(sc.fit_transform(x_test), columns=X.columns)

## Learning and Params Tuning

In [24]:
cv_params = {
    'k': 5,
    'cat_features': None, 
    'do_target_encoding': False, 
    'postprocess': None, 
    'classification': False,
    'split_seed': 2019
}

In [ ]:
rf = model_tuna(RandomForestClassifier(), X, y, 
                 eval_metric=AUC, inverse_metric=True,
                 cv_params=cv_params, n_trials=100,
                 thread_count=-2, seed=2019, verbose=True, return_model=True)

In [ ]:
cat = model_tuna(CatBoostClassifier(), X, y, 
                 eval_metric=AUC, inverse_metric=True,
                 def_params={'eval_metric': 'AUC', 'max_iteration': 10000}, 
                 fit_params={'early_stopping_rounds': 1000},
                 cv_params=cv_params, n_trials=100,
                 thread_count=-2, seed=2019, verbose=True, return_model=True)

In [25]:
# Taken from Gabriel Preda's kernel
LGBM_DEF_PARAMS =  {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbose': -1,
    'num_rounds': 100000
}

LGBM_FIT_PARAMS = {'verbose': 0, 'early_stopping_rounds': 1000}

In [ ]:
lgbc = model_tuna(LGBMClassifier(), X, y, 
                  eval_metric=AUC, inverse_metric=True,
                  def_params=LGBM_DEF_PARAMS, fit_params=LGBM_FIT_PARAMS,
                  cv_params=cv_params, n_trials=100,
                  thread_count=-2, seed=2019, verbose=True, return_model=True)

In [87]:
test_params = {}
lgbc_params = dict(test_params, **LGBM_DEF_PARAMS)
lgbc = LGBMClassifier(**lgbc_params)

In [105]:
def shuffle_wrapper(x_train, y_train, x_test, y_test):
    return shuffle_augmentation(x_train, y_train, x_test, y_test, multiplier=(1,2))

In [126]:
x_train, y_train, x_test, y_test = unique_features(train_data.values[0:150000, 1:], train_data.values[0:150000, 0], train_data.values[150000:, 1:], train_data.values[150000:, 0])

new_features: 400


In [139]:
lgbc.set_params(n_jobs=6)
y_pred = cross_validate(lgbc, X, y, eval_metric=AUC, 
                        fit_params=LGBM_FIT_PARAMS, verbose=True, 
#                         postprocess=unique_features, 
#                         cat_features=CAT_FEATURES_IDX, do_target_encoding=True, 
#                         prediction=True, X_test=test_data.values,
                       )

Model name: LGBMClassifier
...score: 0.898368


KeyboardInterrupt: 

In [94]:
lgbc.fit(x_train, y_train, eval_set=(x_test, y_test), **LGBM_FIT_PARAMS)

LGBMClassifier(bagging_fraction=0.4, bagging_freq=5, boost='gbdt',
        boost_from_average='false', boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.05,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        metric='auc', min_child_samples=20, min_child_weight=0.001,
        min_data_in_leaf=80, min_split_gain=0.0,
        min_sum_hessian_in_leaf=10.0, n_estimators=100, n_jobs=6,
        num_leaves=13, num_rounds=100000, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
        tree_learner='serial', verbose=-1)

In [ ]:
importance = pd.DataFrame({'imp': lgbc.feature_importances_, 'col': X.columns})
_ = importance.plot(kind='barh', x='col', y='imp', figsize=(10, 100))

## Chunk Learning

In [16]:
def chunk_learning(X, y, test, chunk_size=1, tuning=True):
    chunk_idx = np.arange(0, X.shape[1], chunk_size)
    self_stack = {}
    y_stack = {}
    models = {}
    for idx in tqdm(chunk_idx):
        print(idx)
        train_chunk = X.iloc[:, idx:idx+chunk_size]
        test_chunk = test.iloc[:, idx:idx+chunk_size]
        model = LGBMClassifier(**{
            'task': 'train', 'max_depth': 1, 'boosting_type': 'gbdt',
            'objective': 'binary', 'num_leaves': 3, 'learning_rate': 0.1,
            'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'bagging_freq': 5,
            'min_data_in_leaf': 10, 
#             'lambda_l1': 1, 'lambda_l2': 60, 
            'n_jobs': 8, 'verbose': -99
        })
        self_pred, y_pred = cross_validate(model, train_chunk, y, eval_metric=AUC, 
                                           fit_params={'verbose': 0, 'early_stopping_rounds': 50}, verbose=True, 
                                           self_prediction=True, k=5, 
#                                            postprocess=shuffle_augmentation, 
                                           prediction=True, X_test=test_chunk.values)
        self_stack[idx] = self_pred
        y_stack[idx] = y_pred
        models[idx] = model
#         clear_output()
    return self_stack, y_stack, models

In [ ]:
train_stack, test_stack, chunk_models = chunk_learning(X, y, test_data, chunk_size=1)
train_stack = pd.DataFrame().from_dict(train_stack, orient='columns')
test_stack = pd.DataFrame().from_dict(test_stack, orient='columns')

In [29]:
X['max_stack'] = train_stack.max(axis=1).values
X['mean_stack'] = train_stack.mean(axis=1).values
X['product_stack'] = (train_stack * 10).product(axis=1).values

In [32]:
test_data['max_stack'] = test_stack.max(axis=1).values
test_data['mean_stack'] = test_stack.mean(axis=1).values
test_data['product_stack'] = (test_stack * 10).product(axis=1).values

In [ ]:
test_data

In [33]:
# simple sum
print(AUC(y, (train_stack * 10).prod(axis=1).values))
y_pred = (test_stack * 10).prod(axis=1).values

0.9033558326039407


In [38]:
# Ridge
y_pred = cross_validate(Ridge(alpha=0), train_stack, y, eval_metric=AUC, 
                        verbose=True, prediction=True, X_test=test_stack)

Model name: Ridge
...score: 0.901169
...score: 0.897168
...score: 0.902672
...score: 0.901227
...score: 0.904078
CV score: 0.901263 +- 0.002310


In [154]:
ridge = Ridge()
ridge.fit(train_stack, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
importance = pd.DataFrame({'imp': ridge.coef_, 'col': X.columns})
_ = importance.plot(kind='barh', x='col', y='imp', figsize=(10, 100))

# Submission

In [85]:
submission = pd.read_csv(os.path.join(dir_data, 'sample_submission.csv'), index_col=0)
# submission.target = y_pred
submission = pd.merge(submission.drop('target', axis=1), pred_with_id, left_index=True, right_index=True, how='left')
submission = submission.fillna(0)
submission.to_csv(os.path.join(dir_data, 'submission.csv'))

In [61]:
y_pred

array([0.08615607, 0.19842205, 0.18825818, ..., 0.00572141, 0.07053014,
       0.04228766])

In [77]:
pred_with_id = pd.Series(y_pred, test_data.index)
pred_with_id.name = 'target'